In [1]:
import os, sys, json, shutil, argparse

import os.path as osp

parser = argparse.ArgumentParser()

##folder to take experiments from
parser.add_argument("-f", "--f", type=str, required=True)
parser.add_argument("-gpu", "--gpu", type=str, required=False)

##should be implemented in the slurm script --------------------------------

# parser.add_argument("-gpu", "--gpu", type=str, required=False)
# parser.add_argument("-cpus", "--cpus", type=str, required=False)


## slurm  ------------------------------------------------------------------

os.chdir(osp.expanduser("~/work/GraphMerge"))


exp0_folder = 'exp_test'

##########################################################
#      Loop over JSON files and train models             # 
##########################################################

# Generate list over experiments to run
from dev.utils import list_experiments
from dev.train_script_cpu import train_model
    # from dev.train_script_cpu_debug import train_model



#### --------------------------------- ####
# implement for slurm script and slurm .out
# clean_done(exp0_folder)
#### --------------------------------- ####

exp_folder, exp_list = list_experiments(exp0_folder)
print('whoops')

print(f"Starting process with {len(exp_list)} experiments" )
print(exp_list)
# Loop over the experiments
for i, experiment in enumerate(exp_list):
    print('Running ' + experiment[:-5])
    # Load construction dictionary from json file
    with open(osp.join(exp_folder, experiment)) as file:
        construct_dict = json.load(file)
    construct_dict['experiment_name']=experiment[:-5]


    print(f"Starting experiment from {experiment[:-5]}")

    train_model(construct_dict)

    # print(f'Exited training after {epochexit} epochs')
    if construct_dict['move']:
        shutil.move(osp.join(exp_folder, experiment), osp.join(exp0_folder+"/done", experiment))
    print(f"Experiment {experiment[:-5]} done \t {experiment}: {i + 1} / {len(exp_list)}")


whoops
Starting process with 1 experiments
['test_tboard.json']
Running test_tboard
Starting experiment from test_tboard
{'experiment': 'GraphMerge', 'group': 'test1', 'move': False, 'model': 'Sage', 'log': True, 'run_params': {'n_epochs': 1000, 'n_trials': 2, 'batch_size': 128, 'val_epoch': 2, 'early_stopping': True, 'patience': 10, 'loss_func': 'L2', 'metrics': 'scatter', 'performance_plot': 'SAM_base', 'shuffle': True, 'save': True, 'seed': True}, 'learn_params': {'learning_rate': 0.003, 'schedule': 'warmup_exp', 'g_up': 1.2, 'g_down': 0.96, 'warmup': 3}, 'hyper_params': {'hidden_channels': 32, 'conv_layers': 3, 'conv_activation': 'relu', 'decode_layers': 1, 'layernorm': True}, 'data_params': {'case': 'test_all_smass', 'split': 0.8}, 'experiment_name': 'test_tboard'}
RelU conv activation
Made folder for saving model


TypeError: unsupported operand type(s) for *: 'float' and 'dict'

In [44]:
import os, sys, tqdm, json, shutil, glob, argparse

import os.path as osp

import numpy as np

##########################################################
#      Loop over JSON files and train models             # 
##########################################################

# Generate list over experiments to run
from dev.utils import perms
from dev.train_script_cpu import train_model



In [66]:
N_parallel=20
N=200
# Load construction dictionary from json file
with open("exp_compare/diff0.json") as file:
    diff = json.load(file)
    
keys=list(diff.keys())
exp_list=perms(diff)
# Load construction dictionary from json file
with open("exp_compare/base0.json") as file:
    base = json.load(file)

print(f"Starting process with {len(exp_list)} experiments")
print(diff)
# Loop over the experiments

if len(exp_list)>N:
    print(f'Selected too many compare runs ({len(exp_list)}), running {N} random runs')
    idxs = np.random.permutation(len(exp_list))
    exp_list=exp_list[idxs[:N]]
    
cds=[]
for i in range(len(exp_list)):
    construct_dict=base
#     print('Exploring', keys)
#     print('Currently doing', exp_list[i])
    # if i==0:
    #     construct_dict['data_params']['restart']=True
    # else:
    #     construct_dict['data_params']['restart']=False
    for j, key in enumerate(keys):
        if key in construct_dict['learn_params']:
            typ=type(construct_dict['learn_params'][key])
        if key in construct_dict['run_params']:
            typ=type(construct_dict['run_params'][key])
            construct_dict['run_params'][key]=typ(exp_list[i][j])
        elif key in construct_dict['hyper_params']:
            typ=type(construct_dict['hyper_params'][key])
            construct_dict['hyper_params'][key]=typ(exp_list[i][j])
        elif key in construct_dict['data_params']:
            typ=type(construct_dict['data_params'][key])
            construct_dict['data_params'][key]=typ(exp_list[i][j])
    #make_title
    title=''
    for key, val in zip(keys, exp_list[i]):
        title+=key[:4]+str(val)
    construct_dict['experiment_name']=construct_dict['model']+title
    cds.append(construct_dict)
di=''
for key in keys:
    di+=key[:4]+'_'
dirs=[]

exps=np.split(np.array(cds),N_parallel)
for i in range(N_parallel):
    fn=di+str(i)
    group=construct_dict['group']
    fn=f'exp_sweep/{fn}_{group}'
    if not osp.exists(fn+'/todo'):
        os.makedirs(fn+'/todo')
        os.makedirs(fn+'/done')
    for j in range(len(exps[i])):
        with open(fn+'/todo/'+f'exp{j}.json', 'w') as write:
            json.dump(exps[i][j], write)

Starting process with 7290 experiments
{'learning_rate': [0.0003, 0.001, 0.003, 0.01, 0.03, 0.1], 'g_up': [1.1, 1.3, 1.5], 'g_down': [0.9, 0.95, 0.98], 'warmup': [2, 5, 10], 'conv_layers': [2, 3, 5, 7, 10], 'decode_layers': [1, 2, 3], 'hidden_channels': [16, 32, 64]}
Selected too many compare runs (7290), running 200 random runs


In [ ]:
for d in dirs:
    fn=f'exp_sweep/{d}'
    os.mkdir(fn)
    

In [21]:
with open('exp_compare/test.json', 'w') as write:
    json.dump(construct_dict, write)

In [47]:
len(exp_list[:N])

100